In [2]:
!pip install opencv-python tensorflow keras numpy matplotlib

In [3]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
# Load dataset
df = pd.read_csv('fer2013.csv')

# Extract features and labels
X = df['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.uint8).reshape(48, 48))
y = df['emotion']

# Normalize and expand dimensions for CNN
X = np.stack(X.to_numpy())
X = X / 255.0
X = np.expand_dims(X, -1)

# One-hot encode labels
y = to_categorical(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes in FER-2013
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


In [ ]:
model.save("emotion_model.h5")


In [ ]:
from tensorflow.keras.models import load_model

# Load model
model = load_model("emotion_model.h5")

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Quotes per emotion
quotes = {
    'Happy': "😊 Keep smiling! The world is brighter with you.",
    'Sad': "🌈 Every storm runs out of rain. You're stronger than you think.",
    'Angry': "🧘 Take a deep breath. Peace begins with a smile.",
    'Neutral': "😐 Stay balanced and grounded.",
    'Fear': "🌟 Face your fears, they’re usually smaller than they seem.",
    'Surprise': "🎉 Life is full of surprises. Embrace them!",
    'Disgust': "🤢 Let it go. Clean energy brings peace."
}

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    face = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_resized = cv2.resize(face, (48,48))
    face_norm = face_resized / 255.0
    face_input = np.expand_dims(face_norm, axis=(0, -1))

    pred = model.predict(face_input)
    emotion = emotion_labels[np.argmax(pred)]

    # Show emotion + quote
    quote = quotes.get(emotion, "")
    cv2.putText(frame, f"Emotion: {emotion}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(frame, quote, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    cv2.imshow("Emotion Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
